In [1]:
# Install required libraries
!pip install -q trimesh
!pip install -q xatlas
!pip install -q diffusers transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.0/709.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.2/262.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.6 MB/s eta 0:00:00


In [2]:
# Clone the Shap-E repository
!git clone https://github.com/openai/shap-e.git

# Navigate into the cloned directory
%cd shap-e

# Install Shap-E in editable mode
!pip install -e .

Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 336 (delta 35), reused 13 (delta 13), pack-reused 281 (from 2)
Receiving objects: 100% (336/336), 11.72 MiB | 15.39 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/shap-e
Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-3jgd9p6x/clip_32c7cd676fe641b4856e20b49f86a14e
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-3jgd9p6x/clip_32c7cd676fe641b4856e20b49f86a14e
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
import torch
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import decode_latent_mesh


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [5]:
print("Loading transmitter model...")
xm = load_model('transmitter', device=device)


Loading transmitter model...


/content/shap-e/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/shap-e/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

In [6]:
print("Loading text-to-3D diffusion model (text300M)...")
model = load_model('text300M', device=device)


Loading text-to-3D diffusion model (text300M)...


100%|███████████████████████████████████████| 890M/890M [00:10<00:00, 88.4MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [7]:
print("Loading diffusion configuration...")
diffusion = diffusion_from_config(load_config('diffusion'))

print("All models loaded successfully!")

Loading diffusion configuration...
All models loaded successfully!


In [35]:
# Your text prompt for the 3D model
prompt = input("Enter your desired 3D model prompt (e.g., 'a futuristic spaceship', 'a medieval castle'): ")

batch_size = 1 # Number of models to generate (keep at 1 for faster results)
guidance_scale = 15.0 # Higher values make the model stick closer to the prompt


Enter your desired 3D model prompt (e.g., 'a futuristic spaceship', 'a medieval castle'): a cute red mushroom with white spots


In [43]:
print(f"Generating 3D model for prompt: '{prompt}'")

# Generate 3D latents
latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64, # Number of diffusion steps (more steps = potentially better quality, but slower)
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

print("3D latent generated. Decoding mesh...")

Generating 3D model for prompt: 'a cute red mushroom with white spots'


  0%|          | 0/64 [00:00<?, ?it/s]

3D latent generated. Decoding mesh...


In [44]:
import os
import trimesh

# Create a directory to save the models
output_dir = "generated_3d_models"
os.makedirs(output_dir, exist_ok=True)


In [45]:
for i, latent in enumerate(latents):
    # Decode the latent into a triangle mesh
    t = decode_latent_mesh(xm, latent).tri_mesh()

    # Define filenames
    obj_filename = os.path.join(output_dir, f"generated_model_{i}.obj")
    glb_filename = os.path.join(output_dir, f"generated_model_{i}.glb")

    # Save as OBJ
    with open(obj_filename, 'w') as f:
        t.write_obj(f)
    print(f"Saved OBJ to: {obj_filename}")

    # Convert to GLB using trimesh (GLB is often preferred for game engines)
    # Note: trimesh might require additional dependencies for GLB export,
    # but we installed a basic trimesh. If this fails, focus on OBJ.
    try:
        mesh = trimesh.Trimesh(vertices=t.verts, faces=t.faces)
        mesh.export(glb_filename)
        print(f"Saved GLB to: {glb_filename}")
    except Exception as e:
        print(f"Could not save GLB (might need additional trimesh dependencies or a more complex conversion): {e}")
        print("OBJ file is still available.")

print("\nGeneration and saving complete!")
print("You can find your models in the 'generated_3d_models' folder in the Colab file browser (left sidebar).")
print("Download them to view in a 3D viewer (like Blender, Windows 3D Viewer, or online GLB viewers).")

Saved OBJ to: generated_3d_models/generated_model_0.obj
Saved GLB to: generated_3d_models/generated_model_0.glb

Generation and saving complete!
You can find your models in the 'generated_3d_models' folder in the Colab file browser (left sidebar).
Download them to view in a 3D viewer (like Blender, Windows 3D Viewer, or online GLB viewers).


In [46]:
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from IPython.display import display

In [47]:
print("Rendering GIF preview...")


size = 64 # Resolution of the rendered images
cameras = create_pan_cameras(size, device)


Rendering GIF preview...


In [48]:
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode='nerf') # 'nerf' or 'stf'
    # Display the GIF widget
    display(gif_widget(images))
    print(f"Preview for generated_model_{i}.obj:")

print("GIF preview complete!")

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAOTe3t/W1uPOzfXKzNbKyfDHyu3GyNbGxenDxefCxOXBw+S/we…

Preview for generated_model_0.obj:
GIF preview complete!


In [ ]:
-